In [1]:
import pandas as pd
from biom import Table
from biom.util import biom_open
import numpy as np
from sklearn.model_selection import train_test_split
np.random.seed(42)
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import metadata
metadata = pd.read_csv('../network/metadata_v0.csv', index_col=0)
tables = {}

# import/match metabolomics
metabolomics = pd.read_csv('../network/GNPS-raw-metabolomics-quant-table.csv', index_col=0)
metabolomics = metabolomics.drop(['row m/z', 'row retention time'], axis=1)
map_names = pd.read_csv('../network/metabolite_sample_names.txt', sep='\t', index_col=0).orig_sample_name.to_dict()
metabolomics.columns = [c.replace(' Peak area','') for c in metabolomics.columns]
metabolomics.index = ['metab_' + str(i) for i in metabolomics.index]
metabolomics = metabolomics[set(metabolomics.columns) & set(map_names.keys())]
metabolomics.columns = [map_names[c] for c in metabolomics.columns]
tables['metabolomics'] = metabolomics.copy()

# import the rest
all_tables = {'mag':'../network/MAG_bact_table.tsv',
            '18S':'../network/18S_ASV_table.tsv',
            '16S':'../network/16S_ASV_table.tsv',
            'gene':'../network/MAG_bact_genes_table.tsv',
            'gene_module':'../network/MAG_bact_gene_modules_table.tsv'}
for tbl_id, tbl_path in all_tables.items():
    tables[tbl_id] = pd.read_csv(tbl_path, sep='\t', index_col=0)
    
# match and filter the tables
def filter_table(df, use_ids):
    df = df[use_ids]
    df = df[df.sum(1) > 0]
    return df

all_shared_samples = set(metadata.index) & set.intersection(*[set(t.columns) for t in tables.values()])
tables = {tblid:filter_table(tbl, all_shared_samples) for tblid, tbl in tables.items()}
metadata = metadata.reindex(all_shared_samples)

# subset the tables to last active time point
#metadata_subset = pd.concat([df[df.add_0c_group == 'active'].sort_values('timepoint').iloc[[-1], :]
#                             for _, df in metadata.groupby('subjects') if 'active' in list(df.add_0c_group)])

metadata_subset = pd.concat([df
                             for _, df in metadata.groupby('subjects') if len(set(df.add_0c_group)) == 4])

tables_subset = {tblid:filter_table(tbl.copy(), metadata_subset.index) for tblid, tbl in tables.items()}
tables_subset = {tblid:Table(tbl.values, tbl.index, tbl.columns) for tblid, tbl in tables_subset.items()}

# train-tests
for i_ in range(10):
    train, test = train_test_split(metadata_subset, test_size=0.25, shuffle=True, random_state=42,
                                   stratify=metadata_subset[['facility']])
    metadata_subset.loc[:, 'traintest_%i' % i_] = 'train'
    metadata_subset.loc[test.index, 'traintest_%i' % i_] = 'test'
    
# save data 
# metadata_subset.to_csv('../network/split-matched-data/metadata.tsv', sep='\t')
# for tblid, tbl in tables_subset.items():
#    with biom_open('../network/split-matched-data/%s.biom' % (tblid), 'w') as f:
#        tbl.to_hdf5(f, "filtered-table-cm")

tables_subset

{'metabolomics': 2333 x 374 <class 'biom.table.Table'> with 104825 nonzero entries (12% dense),
 'mag': 257 x 374 <class 'biom.table.Table'> with 58498 nonzero entries (60% dense),
 '18S': 5473 x 374 <class 'biom.table.Table'> with 114755 nonzero entries (5% dense),
 '16S': 14237 x 374 <class 'biom.table.Table'> with 318875 nonzero entries (5% dense),
 'gene': 2457 x 374 <class 'biom.table.Table'> with 901890 nonzero entries (98% dense),
 'gene_module': 377 x 374 <class 'biom.table.Table'> with 117093 nonzero entries (83% dense)}

In [4]:
for table_key in tables_subset.keys():
    print(table_key)
    table_df = tables_subset[table_key].to_dataframe()
    display(table_df.head())

metabolomics


,soil.hip.CMU.17.10.2017.08.16.day8,soil.hip.CMU.17.10.2017.08.24.day16,soil.hip.CMU.17.10.2017.08.22.day14,soil.hip.CMU.17.10.2017.08.10.day2,soil.hip.CMU.17.10.2017.08.27.day19,soil.hip.CMU.17.10.2017.08.09.day1,soil.hip.CMU.17.10.2017.08.25.day17,soil.hip.CMU.17.10.2017.08.18.day10,soil.hip.CMU.17.10.2017.08.11.day3,soil.hip.CMU.17.10.2017.08.26.day18,...,soil.hip.SHSU.2016.024.2016.05.02.day18,soil.hip.SHSU.2016.024.2016.04.20.day6,soil.hip.SHSU.2016.024.2016.04.23.day9,soil.hip.SHSU.2016.024.2016.05.04.day20,soil.hip.SHSU.2016.024.2016.04.18.day4,soil.hip.SHSU.2016.024.2016.04.19.day5,soil.hip.SHSU.2016.024.2016.04.15.day1,soil.hip.SHSU.2016.024.2016.04.30.day16,soil.hip.SHSU.2016.024.2016.05.03.day19,soil.hip.SHSU.2016.024.2016.04.25.day11
metab_1,40967.6795,42319.4175,70072.184,81871.996,77229.1450,277872.4775,95474.1480,0.0000,0.0000,0.000,...,86450.310,129596.6595,170358.896,0.000,95399.8950,41355.096,143356.1890,62749.8150,66747.4080,68292.3170
metab_2,110861.7365,67727.4455,71150.631,107084.891,182026.8435,198095.0515,154412.1570,66828.6075,85873.9075,84238.526,...,81207.105,178770.4360,128770.079,142298.562,217757.7515,67319.016,184945.3420,141819.4690,105941.0545,87977.1030
metab_3,13365.6975,12050.6455,13047.685,18704.764,32322.6260,53242.1255,26922.9765,11919.6000,0.0000,0.000,...,21248.880,32280.8010,36985.492,0.000,37552.5300,8171.624,42698.2710,20449.5215,19190.3400,13945.1885
metab_4,17938.7485,17283.3390,30297.778,25694.870,28452.5080,84253.7390,34538.1155,0.0000,0.0000,0.000,...,34144.610,38722.0470,55959.490,0.000,26844.2850,21581.432,42622.7825,22647.5615,24935.0520,34079.2225
metab_5,11512.3490,0.0000,19861.751,16922.820,19897.1010,60645.7505,22323.1995,0.0000,0.0000,0.000,...,21142.380,27462.5295,27154.764,0.000,19896.5100,12371.712,31015.8965,15420.0960,17530.0800,15093.8190


mag


,soil.hip.CMU.17.10.2017.08.16.day8,soil.hip.CMU.17.10.2017.08.24.day16,soil.hip.CMU.17.10.2017.08.22.day14,soil.hip.CMU.17.10.2017.08.10.day2,soil.hip.CMU.17.10.2017.08.27.day19,soil.hip.CMU.17.10.2017.08.09.day1,soil.hip.CMU.17.10.2017.08.25.day17,soil.hip.CMU.17.10.2017.08.18.day10,soil.hip.CMU.17.10.2017.08.11.day3,soil.hip.CMU.17.10.2017.08.26.day18,...,soil.hip.SHSU.2016.024.2016.05.02.day18,soil.hip.SHSU.2016.024.2016.04.20.day6,soil.hip.SHSU.2016.024.2016.04.23.day9,soil.hip.SHSU.2016.024.2016.05.04.day20,soil.hip.SHSU.2016.024.2016.04.18.day4,soil.hip.SHSU.2016.024.2016.04.19.day5,soil.hip.SHSU.2016.024.2016.04.15.day1,soil.hip.SHSU.2016.024.2016.04.30.day16,soil.hip.SHSU.2016.024.2016.05.03.day19,soil.hip.SHSU.2016.024.2016.04.25.day11
UTK.bins.58,0.0000,0.0000,0.00000,0.00,209.20645,0.000,0.0000,0.00000,65.55989,0.0000,...,142.309570,878.650630,686.67080,19.652256,7.107021,150.72356,0.00000,0.00000,3.064841,1298.44510
CMU.bins.99,2199.4407,25433.8520,25361.74400,43743.17,11676.85450,25873.525,50860.8900,19159.42400,40582.75000,32022.0860,...,0.000000,2.969697,0.00000,2.966068,0.000000,0.00000,0.00000,0.00000,1.277570,0.00000
SHSU.bins.42,0.0000,1640.2859,1975.70030,0.00,2541.00680,0.000,4193.8228,281.62924,159.25700,1787.9869,...,324.531250,96.144290,638.82790,956.151860,0.000000,0.00000,170.36514,435.79318,24.816874,297.52332
SHSU.bins.192,0.0000,0.0000,0.00000,0.00,137.50116,0.000,0.0000,0.00000,0.00000,0.0000,...,60.521435,266.926420,224.18906,28.350302,383.716950,103.83103,0.00000,0.00000,30.320877,103.84440
SHSU.bins.235,0.0000,0.0000,6.40744,0.00,0.00000,0.000,0.0000,0.00000,0.00000,0.0000,...,0.000000,0.000000,0.00000,20.759691,0.000000,0.00000,0.00000,0.00000,9.754687,0.00000


18S


...

,soil.hip.CMU.17.10.2017.08.16.day8,soil.hip.CMU.17.10.2017.08.24.day16,soil.hip.CMU.17.10.2017.08.22.day14,soil.hip.CMU.17.10.2017.08.10.day2,soil.hip.CMU.17.10.2017.08.27.day19,soil.hip.CMU.17.10.2017.08.09.day1,soil.hip.CMU.17.10.2017.08.25.day17,soil.hip.CMU.17.10.2017.08.18.day10,soil.hip.CMU.17.10.2017.08.11.day3,soil.hip.CMU.17.10.2017.08.26.day18,...,soil.hip.SHSU.2016.024.2016.05.02.day18,soil.hip.SHSU.2016.024.2016.04.20.day6,soil.hip.SHSU.2016.024.2016.04.23.day9,soil.hip.SHSU.2016.024.2016.05.04.day20,soil.hip.SHSU.2016.024.2016.04.18.day4,soil.hip.SHSU.2016.024.2016.04.19.day5,soil.hip.SHSU.2016.024.2016.04.15.day1,soil.hip.SHSU.2016.024.2016.04.30.day16,soil.hip.SHSU.2016.024.2016.05.03.day19,soil.hip.SHSU.2016.024.2016.04.25.day11
fd1796898145db8675c03b6950633f72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
f84cdc0e4358b155e32a2bc031503bba,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
451a305073d6f00797e39d62a1435605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a20f6a222b37b038bff253e426d3b586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
b86e3e53c4997201202adc4480b2643c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


gene


,soil.hip.CMU.17.10.2017.08.16.day8,soil.hip.CMU.17.10.2017.08.24.day16,soil.hip.CMU.17.10.2017.08.22.day14,soil.hip.CMU.17.10.2017.08.10.day2,soil.hip.CMU.17.10.2017.08.27.day19,soil.hip.CMU.17.10.2017.08.09.day1,soil.hip.CMU.17.10.2017.08.25.day17,soil.hip.CMU.17.10.2017.08.18.day10,soil.hip.CMU.17.10.2017.08.11.day3,soil.hip.CMU.17.10.2017.08.26.day18,...,soil.hip.SHSU.2016.024.2016.05.02.day18,soil.hip.SHSU.2016.024.2016.04.20.day6,soil.hip.SHSU.2016.024.2016.04.23.day9,soil.hip.SHSU.2016.024.2016.05.04.day20,soil.hip.SHSU.2016.024.2016.04.18.day4,soil.hip.SHSU.2016.024.2016.04.19.day5,soil.hip.SHSU.2016.024.2016.04.15.day1,soil.hip.SHSU.2016.024.2016.04.30.day16,soil.hip.SHSU.2016.024.2016.05.03.day19,soil.hip.SHSU.2016.024.2016.04.25.day11
5-epimerase;_Xh,135.392995,260.913642,202.585435,379.264076,167.955975,205.197394,223.300341,142.508572,190.294504,278.462790,...,114.551940,96.950857,151.892285,66.850794,109.963684,187.636100,110.385844,71.857700,173.397333,97.792100
6,3.977856,22.697049,51.878586,9.199818,14.536263,45.217563,20.951288,39.602774,21.295457,34.186768,...,102.790038,96.328424,51.760857,78.770129,32.547968,39.189250,32.010699,49.602860,65.131353,135.504306
6-dehydratase_2;_Xh,2258.259123,2979.570123,2818.593437,3255.366485,2586.533306,3150.601461,3233.259170,2908.479649,3484.343439,3125.050442,...,1949.141046,1577.678491,1897.749008,1643.155554,2487.999779,2056.631914,2494.403107,813.886543,2320.515802,2005.085446
AA1,91.294112,247.698567,229.277756,286.625134,203.261158,303.494787,313.248817,228.177489,248.466203,270.540760,...,170.261617,187.560451,61.915949,119.783198,68.003341,74.518934,70.693789,38.676031,150.180395,139.590086
AA12,49.368077,117.171408,59.940950,62.528618,67.267542,70.164802,60.080811,100.154621,55.916960,53.955425,...,92.616026,72.161615,69.651114,19.708604,171.869268,178.879784,114.010936,25.886881,4.633444,103.209961


gene_module


,soil.hip.CMU.17.10.2017.08.16.day8,soil.hip.CMU.17.10.2017.08.24.day16,soil.hip.CMU.17.10.2017.08.22.day14,soil.hip.CMU.17.10.2017.08.10.day2,soil.hip.CMU.17.10.2017.08.27.day19,soil.hip.CMU.17.10.2017.08.09.day1,soil.hip.CMU.17.10.2017.08.25.day17,soil.hip.CMU.17.10.2017.08.18.day10,soil.hip.CMU.17.10.2017.08.11.day3,soil.hip.CMU.17.10.2017.08.26.day18,...,soil.hip.SHSU.2016.024.2016.05.02.day18,soil.hip.SHSU.2016.024.2016.04.20.day6,soil.hip.SHSU.2016.024.2016.04.23.day9,soil.hip.SHSU.2016.024.2016.05.04.day20,soil.hip.SHSU.2016.024.2016.04.18.day4,soil.hip.SHSU.2016.024.2016.04.19.day5,soil.hip.SHSU.2016.024.2016.04.15.day1,soil.hip.SHSU.2016.024.2016.04.30.day16,soil.hip.SHSU.2016.024.2016.05.03.day19,soil.hip.SHSU.2016.024.2016.04.25.day11
Alpha-galactans,782.998889,381.600591,401.340690,433.550879,467.894083,400.177494,496.179214,499.262278,354.259667,395.743360,...,477.017992,266.704640,421.639445,605.228279,485.262938,467.501885,529.486273,197.662532,795.589465,496.119154
Alpha-mannan,1431.464824,417.662459,424.879447,547.178378,542.341542,428.553895,381.313102,460.087141,420.882622,305.072428,...,506.671509,266.799439,398.733616,574.922540,420.996836,504.788099,580.389607,189.526414,1563.690581,776.814393
Arabinan,2832.631584,2275.218609,2210.351578,2081.555404,2815.662921,2183.158881,1775.112034,2607.423559,1958.441509,2233.779562,...,1186.927440,668.485932,1030.535128,1031.205241,957.890073,1455.022048,1171.462087,370.085994,792.188815,1326.340395
Arabinose_cleavage,754.052743,528.754733,520.756609,537.172763,521.047792,481.584265,507.302173,550.523367,573.862551,496.722771,...,399.675998,203.606049,384.027644,430.238412,406.359177,500.291740,507.088592,172.629305,785.297849,422.681683
Beta-galactan_(pectic_galactan),1610.801135,1163.624260,1052.443032,1066.884734,1079.874443,1115.095674,1111.753895,1159.959149,1173.254109,1015.903971,...,760.939593,363.723345,575.626157,1341.643525,539.032025,920.565559,700.099228,171.768099,1109.818706,779.973846


In [18]:
display(metabolomics.head())
print(metabolomics.shape)
display(metadata.head())
print(metadata.shape)
display(metadata_subset.head())
print(metadata_subset.shape)

,skin.hip.SHSU.2016.007.2016.04.19.day5,soil.hip.UTK.K016.12.2017.06.06.day13,skin.hip.SHSU.2016.076.2016.12.08.day17,soil.hip.CMU.16.05.2016.mo.day12,skin.hip.UTK.K016.12.2017.06.05.day12,skin.hip.UTK.K016.04.2016.10.02.day7,soil.hip.CMU.16.15.2016.12.day17,NaN,skin.hip.CMU.16.04.2016.mo.day16,skin.hip.CMU.16.04.2016.mo.day3,...,soil.hip.CMU.17.12.2017.10.27.day18,soil.hip.SHSU.2016.067.2016.09.29.day14,soil.hip.CMU.17.04.2017.03.day20,NaN,skin.hip.CMU16.15.2017.01.day21,skin.hip.UTK.K016.01.2016.04.10.day6,skin.hip.SHSU.2017.045.2017.12.23.day12,soil.hip.CMU.16.09.2016.07.30.day5,skin.hip.UTK.K016.01.2016.04.20.day16,soil.hip.SHSU.2016.064.2016.09.12.day19
metab_1,158871.7950,10473.0375,179298.0410,112037.2305,198376.116,74712.4035,66960.5700,0.0000,150675.174,274379.8865,...,35707.3620,58849.1425,38239.6850,116851.8925,151473.0500,201226.740,349938.0140,81220.3960,217498.2400,103126.1190
metab_2,212073.4985,26614.9840,211718.8375,150607.5180,188884.356,225981.6640,92532.0855,111684.3860,259870.270,262189.7795,...,78978.7975,94656.5595,49992.5045,326404.9625,254827.1485,225625.574,206177.9790,243058.1950,217967.1735,134703.9300
metab_3,72460.6430,5657.9625,39945.1630,27398.4035,50884.950,53861.3670,19854.8380,12014.1995,55685.133,68002.8485,...,11179.5100,12259.5600,12023.4090,58299.4885,57950.2125,68184.780,82608.1645,29782.0330,92682.2475,19853.5145
metab_4,32689.5045,6472.3500,45096.0440,37877.8270,46728.022,16925.5065,28319.0720,12958.6715,43710.732,62148.2150,...,16777.3840,28053.0825,17137.0310,27312.7605,41674.4075,38273.745,87354.0015,28087.0060,40261.7250,34571.2240
metab_5,29679.1760,4428.1875,42852.0680,24626.0430,39605.188,14625.0615,16425.7960,0.0000,32397.300,36852.3950,...,10184.7560,17671.2275,10952.7400,22897.6590,31313.9375,33606.645,73460.5970,18406.4715,36988.1600,22749.0095


(4284, 1706)


,season,subjects,add_0c,add_0c_group,facility,timepoint
sample_name,,,,,,
soil.hip.UTK.K016.08.2017.Jan.27.day21,winter,K016.08,215.900000,advanced,ARF,21
soil.hip.CMU.17.12.2017.10.19.day10,fall,17-12,107.850000,active,FIRS,10
soil.hip.CMU.16.09.2016.08.05.day11,summer,Mrs. 16-09,269.700000,advanced,FIRS,11
soil.hip.CMU.17.12.2017.10.16.day7,fall,17-12,68.950000,active,FIRS,7
soil.hip.SHSU.2016.011.2016.04.27.day13,spring,shsu.2016.011,271.111111,advanced,STAFS,13


(529, 6)


,season,subjects,add_0c,add_0c_group,facility,timepoint,traintest_0,traintest_1,traintest_2,traintest_3,traintest_4,traintest_5,traintest_6,traintest_7,traintest_8,traintest_9
sample_name,,,,,,,,,,,,,,,,
soil.hip.CMU.17.10.2017.08.11.day3,summer,17-10,49.75,early,FIRS,3,train,train,train,train,train,train,train,train,train,train
soil.hip.CMU.17.10.2017.08.22.day14,summer,17-10,322.80,advanced,FIRS,14,test,test,test,test,test,test,test,test,test,test
soil.hip.CMU.17.10.2017.08.28.day20,summer,17-10,477.80,advanced,FIRS,20,test,test,test,test,test,test,test,test,test,test
soil.hip.CMU.17.10.2017.08.09.day1,summer,17-10,0.00,initial,FIRS,1,train,train,train,train,train,train,train,train,train,train
soil.hip.CMU.17.10.2017.08.23.day15,summer,17-10,348.05,advanced,FIRS,15,train,train,train,train,train,train,train,train,train,train


(374, 16)


In [13]:
print("# unique subjects:",len(metadata.subjects.unique()))
print(metadata.season.value_counts())
print(metadata.facility.value_counts())
#print(metadata.timepoint.value_counts())

# unique subjects: 36
spring    147
winter    130
summer    130
fall      122
Name: season, dtype: int64
ARF      211
STAFS    168
FIRS     150
Name: facility, dtype: int64
